In [1]:
from model import *
from data import *
import cv2

Using TensorFlow backend.


In [2]:
model_checkpoint = ModelCheckpoint('unet_weights.hdf5', monitor='loss',verbose=1, save_best_only=True)
model = unet()

## Train dengan data Generator

In [ ]:
data_gen_args = dict()#rotation_range=0.2,
                    #width_shift_range=0.05,
                    #height_shift_range=0.05,
                    #shear_range=0.05,
                    #zoom_range=0.05,
                    #horizontal_flip=True,
                    #fill_mode='nearest')
myGene = trainGenerator(2,'data/train','image','label',data_gen_args,save_to_dir = None)
model.fit_generator(myGene, steps_per_epoch = 300, epochs = 10, callbacks = [model_checkpoint])

## Train dengan npy file

In [3]:
imgs_train, imgs_mask_train = geneTrainNpy("data/train/aug/","data/train/aug/")
model.fit(imgs_train, imgs_mask_train, batch_size = 2, epochs = 10, verbose = 1, validation_split= 0.2, shuffle = True, callbacks = [model_checkpoint])
#model.fit(imgs_train, imgs_mask_train, batch_size = 2,epochs = 10, verbose = 1, shuffle = True, validation_data=0.2, callbacks = [model_checkpoint])

Train on 480 samples, validate on 120 samples
Epoch 1/10
480/480 [==============================] - 267s 557ms/step - loss: 0.0701 - accuracy: 0.9794 - val_loss: 0.0487 - val_accuracy: 0.9843

Epoch 00001: loss improved from inf to 0.07013, saving model to unet_weights.hdf5
Epoch 2/10
480/480 [==============================] - 355s 740ms/step - loss: 0.0343 - accuracy: 0.9794 - val_loss: 0.0177 - val_accuracy: 0.9843

Epoch 00002: loss improved from 0.07013 to 0.03428, saving model to unet_weights.hdf5
Epoch 3/10
480/480 [==============================] - 337s 702ms/step - loss: 0.0251 - accuracy: 0.9919 - val_loss: 0.0144 - val_accuracy: 0.9982

Epoch 00003: loss improved from 0.03428 to 0.02514, saving model to unet_weights.hdf5
Epoch 4/10
480/480 [==============================] - 288s 601ms/step - loss: 0.0214 - accuracy: 0.9958 - val_loss: 0.0226 - val_accuracy: 0.9936

Epoch 00004: loss improved from 0.02514 to 0.02142, saving model to unet_weights.hdf5
Epoch 5/10
480/480 [======

## Simpan Data Prediksi

In [4]:
from data import *
testGene = testGenerator('data/test')
results = cv2.normalize(src=model.predict_generator(testGene,20,verbose=1), dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
saveResult("data/test", results)

20/20 [==============================] - 3s 148ms/step
